In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('Dataset/training_set.csv', index_col=0)

In [3]:
dataset.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,
1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,...,1,1,0,0,0,1,1,0,Normal,0
2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,...,1,2,0,0,0,1,6,0,Normal,0
3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,...,1,3,0,0,0,2,6,0,Normal,0
4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,...,1,3,1,1,0,2,1,0,Normal,0
5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,...,1,40,0,0,0,2,39,0,Normal,0


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175341 entries, 1 to 175341
Data columns (total 44 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   dur                175341 non-null  float64
 1   proto              175341 non-null  object 
 2   service            175341 non-null  object 
 3   state              175341 non-null  object 
 4   spkts              175341 non-null  int64  
 5   dpkts              175341 non-null  int64  
 6   sbytes             175341 non-null  int64  
 7   dbytes             175341 non-null  int64  
 8   rate               175341 non-null  float64
 9   sttl               175341 non-null  int64  
 10  dttl               175341 non-null  int64  
 11  sload              175341 non-null  float64
 12  dload              175341 non-null  float64
 13  sloss              175341 non-null  int64  
 14  dloss              175341 non-null  int64  
 15  sinpkt             175341 non-null  float64
 16  di

In [5]:
dataset.duplicated().sum()

67601

In [6]:
dataset.drop_duplicates(inplace=True)

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107740 entries, 1 to 175338
Data columns (total 44 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   dur                107740 non-null  float64
 1   proto              107740 non-null  object 
 2   service            107740 non-null  object 
 3   state              107740 non-null  object 
 4   spkts              107740 non-null  int64  
 5   dpkts              107740 non-null  int64  
 6   sbytes             107740 non-null  int64  
 7   dbytes             107740 non-null  int64  
 8   rate               107740 non-null  float64
 9   sttl               107740 non-null  int64  
 10  dttl               107740 non-null  int64  
 11  sload              107740 non-null  float64
 12  dload              107740 non-null  float64
 13  sloss              107740 non-null  int64  
 14  dloss              107740 non-null  int64  
 15  sinpkt             107740 non-null  float64
 16  di

In [8]:
# check for extraneous values in each column

for col in dataset.columns:
    print(f"Column: {col}, Unique Values: {dataset[col].value_counts()} \n")

Column: dur, Unique Values: 0.000009    4276
0.000003    2408
0.000008    2243
0.000010    1662
0.000005    1639
            ... 
0.694603       1
1.854685       1
1.315859       1
1.131023       1
1.208004       1
Name: dur, Length: 74039, dtype: int64 

Column: proto, Unique Values: tcp     76293
udp     23027
unas     2582
ospf      775
arp       633
        ...  
tlsp       26
ggp        24
igmp       17
icmp       13
rtp         1
Name: proto, Length: 133, dtype: int64 

Column: service, Unique Values: -           64951
http        18135
dns         11046
smtp         5025
ftp-data     3282
ftp          2691
ssh          1295
pop3         1104
snmp           63
ssl            56
dhcp           55
irc            25
radius         12
Name: service, dtype: int64 

Column: state, Unique Values: FIN    74478
INT    19726
CON    12487
REQ      953
RST       83
ECO       10
no         1
URN        1
PAR        1
Name: state, dtype: int64 

Column: spkts, Unique Values: 2       26233
10  

In [9]:
# replace the - in the 'service' column with 'no_service'
dataset['service'] = dataset['service'].replace('-', 'no_service')
dataset['service'].value_counts()

no_service    64951
http          18135
dns           11046
smtp           5025
ftp-data       3282
ftp            2691
ssh            1295
pop3           1104
snmp             63
ssl              56
dhcp             55
irc              25
radius           12
Name: service, dtype: int64

The 'is_ftp_login' column is a binary column but has more than two unique values. We'll drop these extras

In [10]:
# drop the rows having '4' and '2' in the 'is_ftp_login' column
dataset = dataset[~dataset['is_ftp_login'].isin([4, 2])]

In [11]:
dataset.shape

(107731, 44)

In [12]:
dataset.describe()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
count,107731.000000,107731.000000,107731.000000,1.077310e+05,1.077310e+05,1.077310e+05,107731.000000,107731.000000,1.077310e+05,1.077310e+05,...,107731.000000,107731.000000,107731.000000,107731.000000,107731.000000,107731.000000,107731.000000,107731.000000,107731.000000,107731.000000
mean,1.354659,30.191347,30.060094,1.382647e+04,2.373436e+04,3.869417e+04,143.053958,121.591223,4.968532e+07,1.089326e+06,...,1.984053,1.441925,4.025397,0.017052,0.017052,0.182000,3.905598,4.785187,0.005059,0.518347
std,5.515859,173.425077,136.551577,2.226440e+05,1.786282e+05,1.166361e+05,107.777127,116.014892,2.114147e+08,3.013890e+06,...,3.348072,2.096089,6.156377,0.129465,0.129465,0.578816,4.987496,6.197879,0.070946,0.499666
min,0.000000,1.000000,0.000000,2.800000e+01,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,0.001701,4.000000,2.000000,5.000000e+02,1.780000e+02,2.623731e+01,31.000000,29.000000,9.320282e+03,2.113422e+03,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
50%,0.304939,10.000000,8.000000,9.520000e+02,3.540000e+02,8.729732e+01,62.000000,29.000000,8.766153e+04,7.914065e+03,...,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,2.000000,3.000000,0.000000,1.000000
75%,0.971046,22.000000,18.000000,2.542000e+03,3.080000e+03,4.020678e+03,254.000000,252.000000,1.115407e+06,5.463986e+05,...,2.000000,1.000000,4.000000,0.000000,0.000000,0.000000,4.000000,6.000000,0.000000,1.000000
max,59.999989,9616.000000,10974.000000,1.296523e+07,1.465555e+07,1.000000e+06,255.000000,254.000000,5.988000e+09,2.242273e+07,...,51.000000,46.000000,65.000000,1.000000,1.000000,30.000000,60.000000,62.000000,1.000000,1.000000


In [13]:
dataset

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,
1,0.121478,tcp,no_service,FIN,6,4,258,172,74.087490,252,...,1,1,0,0,0,1,1,0,Normal,0
2,0.649902,tcp,no_service,FIN,14,38,734,42014,78.473372,62,...,1,2,0,0,0,1,6,0,Normal,0
3,1.623129,tcp,no_service,FIN,8,16,364,13186,14.170161,62,...,1,3,0,0,0,2,6,0,Normal,0
4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,...,1,3,1,1,0,2,1,0,Normal,0
5,0.449454,tcp,no_service,FIN,10,6,534,268,33.373826,254,...,1,40,0,0,0,2,39,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175268,1.914309,tcp,smtp,FIN,74,30,69997,2132,53.805315,62,...,1,1,0,0,0,18,1,0,Exploits,1
175278,3.719110,tcp,no_service,FIN,66,340,3086,426483,108.897021,62,...,1,1,0,0,0,2,1,0,Exploits,1
175300,0.996503,tcp,pop3,FIN,20,30,942,16684,49.171955,62,...,1,1,0,0,0,7,1,0,Exploits,1
